In [10]:
!unzip llama_3b_3_2.zip

Archive:  llama_3b_3_2.zip
   creating: llama3.2-3b-qlora-summary/
  inflating: llama3.2-3b-qlora-summary/tokenizer.json  
  inflating: llama3.2-3b-qlora-summary/adapter_config.json  
  inflating: llama3.2-3b-qlora-summary/tokenizer_config.json  
  inflating: llama3.2-3b-qlora-summary/README.md  
  inflating: llama3.2-3b-qlora-summary/chat_template.jinja  
  inflating: llama3.2-3b-qlora-summary/special_tokens_map.json  
   creating: llama3.2-3b-qlora-summary/checkpoint-300/
  inflating: llama3.2-3b-qlora-summary/checkpoint-300/tokenizer.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-300/adapter_config.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-300/scheduler.pt  
  inflating: llama3.2-3b-qlora-summary/checkpoint-300/tokenizer_config.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-300/README.md  
  inflating: llama3.2-3b-qlora-summary/checkpoint-300/chat_template.jinja  
  inflating: llama3.2-3b-qlora-summary/checkpoint-300/special_tokens_map.json  
  inf

In [1]:
!pip install pandas datasets

In [2]:
!pip install transformers torch

In [3]:
!pip install xformers trl peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
import json
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from torch.utils.data import DataLoader
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch
import matplotlib.pyplot as plt

# Loading the Dataset

In [5]:
# Load JSONL data (Custom Dataset)
print("\n2-[1/8] Loading dataset...")
data = []
with open("custom_dataset.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

custom_dataset = Dataset.from_dict({
    "messages": [item["messages"] for item in data]
})

print(f"✓ Loaded {len(custom_dataset)} examples")


2-[1/8] Loading dataset...
✓ Loaded 1004 examples


# Preparing the Train and the Test set

Split 0.1 (90% - Train and 10% Test)

In [7]:
# Split dataset
split_dataset = custom_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]
print(f"✓ Train: {len(train_dataset)} | Val: {len(val_dataset)}")

✓ Train: 903 | Val: 101


# Loading the FineTuned Model

Loading the Finetuned (with TL;DR Dataset) LLma 3b Model

In [6]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `YTA` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggin

In [8]:
print("\n[2/8] Configuring 8-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16,
)


[2/8] Configuring 8-bit quantization...


In [9]:
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
ADAPTER_PATH = "./llama3.2-3b-qlora-summary"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config,  # SAME as original
)

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
model = PeftModel.from_pretrained(
    model,
    ADAPTER_PATH,
    is_trainable=True   # to continue training
)

model.train()
model.config.use_cache = False

print("✓ Fine-tuned model loaded")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

✓ Fine-tuned model loaded


In [13]:
model.print_trainable_parameters()

# Tokenization function with proper chat formatting
print("\n[5/8] Preparing tokenization...")

trainable params: 24,313,856 || all params: 3,237,063,680 || trainable%: 0.7511

[5/8] Preparing tokenization...


In [14]:
model.peft_config  # for verifying lora config

{'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, peft_version='0.18.0', base_model_name_or_path='meta-llama/Llama-3.2-3B-Instruct', revision=None, inference_mode=False, r=16, target_modules={'q_proj', 'k_proj', 'o_proj', 'down_proj', 'gate_proj', 'up_proj', 'v_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None, arrow_config=None, ensure_weight_tying=False)}

In [15]:
def format_and_tokenize(example, force=4096):
    # Apply LLaMA chat template
    formatted_text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )

    # Tokenize (NO padding, NO manual labels)
    tokenized = tokenizer(
        formatted_text,
        truncation=True,
        max_length=force,   # IMPORTANT: increase for long transcripts
        padding=False,
    )

    return tokenized

## Note

we need to set the max_length for the tokenization function to 10_000 for the custom dataset

In [16]:
format_and_tokenize_for_cds = lambda x: format_and_tokenize(x, force=10_000)

In [17]:
# Apply tokenization
print("✓ Tokenizing train dataset...")
train_dataset = train_dataset.map(
    format_and_tokenize_for_cds,
    remove_columns=["messages"],
    batched=False,
    desc="Tokenizing train"
)


✓ Tokenizing train dataset...


Tokenizing train:   0%|          | 0/903 [00:00<?, ? examples/s]

In [18]:
print("✓ Tokenizing validation dataset...")
val_dataset = val_dataset.map(
    format_and_tokenize_for_cds,
    remove_columns=["messages"],
    batched=False,
    desc="Tokenizing validation"
)

✓ Tokenizing validation dataset...


Tokenizing validation:   0%|          | 0/101 [00:00<?, ? examples/s]

In [19]:
# Show sample stats (pre-training)
sample = train_dataset[0]

print("\n✓ Sample stats:")
print(f"  - Input length: {len(sample['input_ids'])} tokens")
print(f"  - Attention tokens: {sum(sample['attention_mask'])} tokens")
print(f"  - Truncated: {'Yes' if len(sample['input_ids']) == 2 ** 15 else 'No'}")



✓ Sample stats:
  - Input length: 1382 tokens
  - Attention tokens: 1382 tokens
  - Truncated: No


In [20]:
print("\n[6/8] Configuring training...")
training_args = TrainingArguments(
    output_dir="./final-summary",
    num_train_epochs=3,

    # Memory-optimized batch settings
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,

    # Learning settings
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=100,
    max_grad_norm=1.0,

    # Evaluation and saving
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,

    # Optimization
    bf16=True,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,

    # Efficiency
    group_by_length=True,
    dataloader_num_workers=4,

    # Logging
    logging_steps=10,
    report_to="none",
)


[6/8] Configuring training...


In [21]:
print("✓ Training configuration:")
print(f"  - Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Total training steps: ~{len(train_dataset) * 3 // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")
print(f"  - Learning rate: {training_args.learning_rate}")

✓ Training configuration:
  - Effective batch size: 16
  - Total training steps: ~169
  - Learning rate: 0.0002


In [22]:
# Data collator with dynamic padding
print("\n[7/8] Creating data collator...")
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8  # Efficient padding for GPU
)


[7/8] Creating data collator...


In [23]:
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)


In [24]:
# Start training
print("\n[8/8] Starting training...")
print("=" * 60)
print("TRAINING IN PROGRESS")
print("=" * 60)

trainer.train()


[8/8] Starting training...
TRAINING IN PROGRESS


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.998300,2.143472


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=171, training_loss=2.1060634607460065, metrics={'train_runtime': 6778.3477, 'train_samples_per_second': 0.4, 'train_steps_per_second': 0.025, 'total_flos': 2.0566782399297946e+17, 'train_loss': 2.1060634607460065, 'epoch': 3.0})

In [25]:
# Save final model
print("\n" + "=" * 60)
print("TRAINING COMPLETE")
print("=" * 60)
print("\nSaving model...")
trainer.save_model("./final-summary")
tokenizer.save_pretrained("./final-summary")


TRAINING COMPLETE

Saving model...


('./final-summary/tokenizer_config.json',
 './final-summary/special_tokens_map.json',
 './final-summary/chat_template.jinja',
 './final-summary/tokenizer.json')

In [26]:
!zip -r ./final-summary.zip ./final-summary

  adding: final-summary/ (stored 0%)
  adding: final-summary/README.md (deflated 65%)
  adding: final-summary/checkpoint-100/ (stored 0%)
  adding: final-summary/checkpoint-100/README.md (deflated 65%)
  adding: final-summary/checkpoint-100/adapter_config.json (deflated 58%)
  adding: final-summary/checkpoint-100/training_args.bin (deflated 53%)
  adding: final-summary/checkpoint-100/special_tokens_map.json (deflated 63%)
  adding: final-summary/checkpoint-100/tokenizer_config.json (deflated 96%)
  adding: final-summary/checkpoint-100/rng_state.pth (deflated 26%)
  adding: final-summary/checkpoint-100/trainer_state.json (deflated 70%)
  adding: final-summary/checkpoint-100/chat_template.jinja (deflated 71%)
  adding: final-summary/checkpoint-100/optimizer.pt (deflated 11%)
  adding: final-summary/checkpoint-100/scheduler.pt (deflated 62%)
  adding: final-summary/checkpoint-100/adapter_model.safetensors (deflated 7%)
  adding: final-summary/checkpoint-100/tokenizer.json (deflated 85%)
 